### Setup e Variáveis

In [ ]:
from notebooks.core.bemol_lakestorage import BemolLakeStorage
from core.bemol_controller import BemolController
from core.bemol_landing_reader import BemolLandingReader
from core.bemol_logger import BemolLogger
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col

In [ ]:
# Instanciando classe BemolLogger
logger = BemolLogger("bronze_products_carts")

# Configurando Spark com Delta Lake
spark = (
  SparkSession.builder
  .appName("IngestaoBronzeProdutos")
  .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
  .getOrCreate()
)

# Instanciando classe de leitura/escrita de dados BemolLakeStorage
lakehouse = BemolLakeStorage(spark, logger)

# Instanciando classe de leitura da camada landing BemolLandingReader
landing = BemolLandingReader(logger)

In [ ]:
# Definindo URL's da API e destinos dos dados
url_products = "https://fakestoreapi.com/products"
url_carts = "https://fakestoreapi.com/carts"
destination_path_products = "../data/bronze/products"
destination_path_carts = "../data/bronze/carts"

# Path para salvar os dados de monitoramento
destination_path_monitor = "../data/monitoring/"

### Leitura

In [ ]:
# Lendo dados da API e criando DataFrame através do método read_api
df_carts = landing.read_api(spark, url_carts)
df_products = landing.read_api(spark, url_products)

### Transformações Produto

In [ ]:
# Remove registros com valores nulos em colunas críticas
df_products_bronze = df_products.dropna(subset=["id"])

# Renomeia coluna pora manter evitar ambiguidade
df_products_bronze = df_products_bronze.withColumnRenamed("title", "product_title")

# Extrai campos aninhados
df_products_bronze = (
  df_products_bronze
  .withColumn("rating_count", col("rating.count"))
  .withColumn("rating", col("rating.rate"))
  )

### Tranformações Carrinho

In [ ]:
# Remove linhas com valores nulos em colunas críticas
df_carts_bronze = df_carts.dropna(subset=["id", "products"])

# Explode a coluna de produtos para criar uma linha por produto em cada carrinho
df_carts_bronze = df_carts_bronze.withColumn("products", explode("products"))

# Extrai campos aninhados e renomeia colunas para evitar ambiguidade
df_carts_bronze = df_carts_bronze.select(
    col("id").alias("cart_id"),
    col("userId").alias("user_id"),
    col("date").alias("cart_date"),
    col("products.productId").alias("product_id"),
    col("products.quantity").alias("product_quantity")
)

### Escrita

In [ ]:
# Adiciona coluna de controle através do método control_field
df_products_bronze = BemolController.control_field(df_products_bronze, layer="bronze")
df_carts_bronze = BemolController.control_field(df_carts_bronze, layer="bronze")

In [ ]:
# Grava os dados na camada Bronze no formato Delta, utilizando o modo overwrite como padrão através do método write_bronze
lakehouse.write_bronze(df_products_bronze, destination_path_products, table_name="bronze_products")
lakehouse.write_bronze(df_carts_bronze, destination_path_carts, table_name="bronze_carts")

In [ ]:
# Grava os dados de monitoramento no formato Delta, utilizando o modo overwrite como padrão através do método export_delta
lakehouse.monitor.export_delta(spark, destination_path_monitor)